In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ಹಂತ 1: ರಚಿತ ಔಟ್‌ಪುಟ್‌ಗಳಿಗಾಗಿ ಪೈಡ್ಯಾಂಟಿಕ್ ಮಾದರಿಗಳನ್ನು ನಿರ್ಧರಿಸಿ

ಈ ಮಾದರಿಗಳು ಏಜೆಂಟ್‌ಗಳು ಹಿಂತಿರುಗಿಸುವ **schema** ಅನ್ನು ನಿರ್ಧರಿಸುತ್ತವೆ. ಪೈಡ್ಯಾಂಟಿಕ್‌ನೊಂದಿಗೆ `response_format` ಬಳಸುವುದರಿಂದ ಖಚಿತವಾಗುತ್ತದೆ:
- ✅ ಪ್ರಕಾರ-ಸುರಕ್ಷಿತ ಡೇಟಾ ಹೊರತೆಗೆಯುವುದು
- ✅ ಸ್ವಯಂಚಾಲಿತ ಮಾನ್ಯತೆ
- ✅ ಉಚಿತ-ಪಠ್ಯ ಪ್ರತಿಸ್ಪಂದನಗಳಿಂದ ಪಾರ್ಸಿಂಗ್ ದೋಷಗಳಿಲ್ಲ
- ✅ ಕ್ಷೇತ್ರಗಳ ಆಧಾರದ ಮೇಲೆ ಸುಲಭವಾದ ಷರತ್ತುಮೂಲಕ ಮಾರ್ಗಸೂಚಿ


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ಹಂತ 2: ಹೋಟೆಲ್ ಬುಕ್ಕಿಂಗ್ ಸಾಧನವನ್ನು ರಚಿಸಿ

ಈ ಸಾಧನವನ್ನು **availability_agent** ಕೋಣೆಗಳು ಲಭ್ಯವಿದೆಯೇ ಎಂದು ಪರಿಶೀಲಿಸಲು ಕರೆಮಾಡುತ್ತದೆ. ನಾವು `@ai_function` ಅಲಂಕಾರವನ್ನು ಬಳಸುತ್ತೇವೆ:
- ಪೈಥಾನ್ ಫಂಕ್ಷನ್ ಅನ್ನು AI-ಕಾಲ್ ಮಾಡಬಹುದಾದ ಸಾಧನವಾಗಿ ಪರಿವರ್ತಿಸಲು
- LLM ಗೆ ಸ್ವಯಂಚಾಲಿತ JSON schema ರಚಿಸಲು
- ಪ್ಯಾರಾಮೀಟರ್ ಮಾನ್ಯತೆ ನಿರ್ವಹಿಸಲು
- ಏಜೆಂಟ್‌ಗಳಿಂದ ಸ್ವಯಂಚಾಲಿತ ಕರೆಯನ್ನು ಸಕ್ರಿಯಗೊಳಿಸಲು

ಈ ಡೆಮೋಗಾಗಿ:
- **ಸ್ಟಾಕ್‌ಹೋಮ್, ಸಿಯಾಟಲ್, ಟೋಕಿಯೋ, ಲಂಡನ್, ಆಮ್‌ಸ್ಟರ್‌ಡ್ಯಾಮ್** → ಕೋಣೆಗಳು ಲಭ್ಯವಿದೆ ✅
- **ಇತರ ಎಲ್ಲಾ ನಗರಗಳು** → ಕೋಣೆಗಳು ಲಭ್ಯವಿಲ್ಲ ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ಹಂತ 3: ಮಾರ್ಗದರ್ಶನಕ್ಕಾಗಿ ಶರತ್ತು ಕಾರ್ಯಗಳನ್ನು ನಿರ್ಧರಿಸಿ

ಈ ಕಾರ್ಯಗಳು ಏಜೆಂಟ್‌ನ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಪರಿಶೀಲಿಸುತ್ತವೆ ಮತ್ತು ಕಾರ್ಯಪ್ರವಾಹದಲ್ಲಿ ಯಾವ ಮಾರ್ಗವನ್ನು ತೆಗೆದುಕೊಳ್ಳಬೇಕೆಂದು ನಿರ್ಧರಿಸುತ್ತವೆ.

**ಮುಖ್ಯ ಮಾದರಿ:**
1. ಸಂದೇಶವು `AgentExecutorResponse` ಆಗಿದೆಯೇ ಎಂದು ಪರಿಶೀಲಿಸಿ
2. ರಚಿತ ಔಟ್‌ಪುಟ್ (Pydantic ಮಾದರಿ) ಅನ್ನು ವಿಶ್ಲೇಷಿಸಿ
3. ಮಾರ್ಗದರ್ಶನವನ್ನು ನಿಯಂತ್ರಿಸಲು `True` ಅಥವಾ `False` ಅನ್ನು ಹಿಂತಿರುಗಿಸಿ

ಕಾರ್ಯಪ್ರವಾಹವು ಈ ಶರತ್ತುಗಳನ್ನು **ಎಡ್ಜ್‌ಗಳು** ಮೇಲೆ ಮೌಲ್ಯಮಾಪನ ಮಾಡುತ್ತದೆ, ಮುಂದಿನ ಯಾವ ಎಕ್ಸಿಕ್ಯೂಟರ್ ಅನ್ನು ಕರೆದೊಯ್ಯಬೇಕೆಂದು ನಿರ್ಧರಿಸಲು.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## ಹಂತ 4: ಕಸ್ಟಮ್ ಡಿಸ್ಪ್ಲೇ ಎಕ್ಸಿಕ್ಯೂಟರ್ ರಚಿಸಿ

ಎಕ್ಸಿಕ್ಯೂಟರ್‌ಗಳು ವರ್ಕ್‌ಫ್ಲೋ ಘಟಕಗಳಾಗಿದ್ದು, ಪರಿವರ್ತನೆಗಳು ಅಥವಾ ಪಾರ್ಶ್ವ ಪರಿಣಾಮಗಳನ್ನು ನಿರ್ವಹಿಸುತ್ತವೆ. ಅಂತಿಮ ಫಲಿತಾಂಶವನ್ನು ಪ್ರದರ್ಶಿಸಲು ನಾವು `@executor` ಡೆಕೊರೇಟರ್ ಅನ್ನು ಬಳಸಿಕೊಂಡು ಕಸ್ಟಮ್ ಎಕ್ಸಿಕ್ಯೂಟರ್ ಅನ್ನು ರಚಿಸುತ್ತೇವೆ.

**ಮುಖ್ಯ ಪರಿಕಲ್ಪನೆಗಳು:**
- `@executor(id="...")` - ಕಾರ್ಯವನ್ನು ವರ್ಕ್‌ಫ್ಲೋ ಎಕ್ಸಿಕ್ಯೂಟರ್ ಆಗಿ ನೋಂದಾಯಿಸುತ್ತದೆ
- `WorkflowContext[Never, str]` - ಇನ್‌ಪುಟ್/ಔಟ್‌ಪುಟ್‌ಗಾಗಿ ಟೈಪ್ ಸೂಚನೆಗಳು
- `ctx.yield_output(...)` - ಅಂತಿಮ ವರ್ಕ್‌ಫ್ಲೋ ಫಲಿತಾಂಶವನ್ನು ನೀಡುತ್ತದೆ


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## ಹಂತ 5: ಪರಿಸರ ವ್ಯತ್ಯಾಸಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ

LLM ಕ್ಲೈಂಟ್ ಅನ್ನು ಸಂರಚಿಸಿ. ಈ ಉದಾಹರಣೆ ಈ ಕೆಳಗಿನವುಗಳೊಂದಿಗೆ ಕೆಲಸ ಮಾಡುತ್ತದೆ:
- **GitHub ಮಾದರಿಗಳು** (GitHub ಟೋಕನ್‌ನೊಂದಿಗೆ ಉಚಿತ ಹಂತ)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ಹಂತ 6: ರಚಿತ ಔಟ್‌ಪುಟ್‌ಗಳೊಂದಿಗೆ AI ಏಜೆಂಟ್‌ಗಳನ್ನು ರಚಿಸಿ

ನಾವು **ಮೂರು ವಿಶೇಷ ಏಜೆಂಟ್‌ಗಳನ್ನು** ರಚಿಸುತ್ತೇವೆ, ಪ್ರತಿ ಒಂದನ್ನು `AgentExecutor` ನಲ್ಲಿ ಸುತ್ತುವರಿಯಲಾಗುತ್ತದೆ:

1. **availability_agent** - ಸಾಧನವನ್ನು ಬಳಸಿಕೊಂಡು ಹೋಟೆಲ್ ಲಭ್ಯತೆ ಪರಿಶೀಲಿಸುತ್ತದೆ
2. **alternative_agent** - ಪರ್ಯಾಯ ನಗರಗಳನ್ನು ಸೂಚಿಸುತ್ತದೆ (ಯಾವುದೇ ಕೊಠಡಿಗಳು ಲಭ್ಯವಿಲ್ಲದಿದ್ದಾಗ)
3. **booking_agent** - ಬುಕಿಂಗ್ ಪ್ರೋತ್ಸಾಹಿಸುತ್ತದೆ (ಕೊಠಡಿಗಳು ಲಭ್ಯವಿದ್ದಾಗ)

**ಮುಖ್ಯ ವೈಶಿಷ್ಟ್ಯಗಳು:**
- `tools=[hotel_booking]` - ಏಜೆಂಟ್‌ಗೆ ಸಾಧನವನ್ನು ಒದಗಿಸುತ್ತದೆ
- `response_format=PydanticModel` - ರಚಿತ JSON ಔಟ್‌ಪುಟ್ ಅನ್ನು ಬಲವಾಗಿ ಅನುಸರಿಸುತ್ತದೆ
- `AgentExecutor(..., id="...")` - ಕಾರ್ಯಪ್ರವಾಹ ಬಳಕೆಗೆ ಏಜೆಂಟ್ ಅನ್ನು ಸುತ್ತುವರಿಯುತ್ತದೆ


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ಹಂತ 7: ಷರತ್ತುಬದ್ಧ ಎಡ್ಜ್‌ಗಳೊಂದಿಗೆ ವರ್ಕ್‌ಫ್ಲೋ ನಿರ್ಮಿಸಿ

ಈಗ ನಾವು `WorkflowBuilder` ಅನ್ನು ಬಳಸಿಕೊಂಡು ಷರತ್ತುಬದ್ಧ ಮಾರ್ಗಸೂಚಿಯೊಂದಿಗೆ ಗ್ರಾಫ್ ಅನ್ನು ನಿರ್ಮಿಸುತ್ತೇವೆ:

**ವರ್ಕ್‌ಫ್ಲೋ ರಚನೆ:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**ಮುಖ್ಯ ವಿಧಾನಗಳು:**
- `.set_start_executor(...)` - ಪ್ರವೇಶ ಬಿಂದುವನ್ನು ಹೊಂದಿಸುತ್ತದೆ
- `.add_edge(from, to, condition=...)` - ಷರತ್ತುಬದ್ಧ ಎಡ್ಜ್ ಅನ್ನು ಸೇರಿಸುತ್ತದೆ
- `.build()` - ವರ್ಕ್‌ಫ್ಲೋವನ್ನು ಅಂತಿಮಗೊಳಿಸುತ್ತದೆ


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## ಹಂತ 8: ಪರೀಕ್ಷಾ ಪ್ರಕರಣ 1 - ಲಭ್ಯತೆ ಇಲ್ಲದ ನಗರ (ಪ್ಯಾರಿಸ್)

ನಮ್ಮ ಸಿಮ್ಯುಲೇಶನ್‌ನಲ್ಲಿ ಯಾವುದೇ ಕೊಠಡಿಗಳು ಇಲ್ಲದ ಪ್ಯಾರಿಸ್‌ನಲ್ಲಿ ಹೋಟೆಲ್‌ಗಳನ್ನು ಕೇಳುವ ಮೂಲಕ **ಲಭ್ಯತೆ ಇಲ್ಲದ** ಮಾರ್ಗವನ್ನು ಪರೀಕ್ಷಿಸೋಣ.


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## ಹಂತ 9: ಪರೀಕ್ಷಾ ಪ್ರಕರಣ 2 - ಲಭ್ಯತೆ ಇರುವ ನಗರ (ಸ್ಟಾಕ್‌ಹೋಮ್)

ಈಗ ನಮ್ಮ ಸಿಮ್ಯುಲೇಶನ್‌ನಲ್ಲಿ ಕೊಠಡಿಗಳು ಇರುವ ಸ್ಟಾಕ್‌ಹೋಮ್‌ನಲ್ಲಿ ಹೋಟೆಲ್‌ಗಳನ್ನು ಕೇಳುವ ಮೂಲಕ **ಲಭ್ಯತೆ** ಮಾರ್ಗವನ್ನು ಪರೀಕ್ಷಿಸೋಣ.


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## ಮುಖ್ಯ ಅಂಶಗಳು ಮತ್ತು ಮುಂದಿನ ಹಂತಗಳು

### ✅ ನೀವು ಕಲಿತದ್ದೇನು:

1. **WorkflowBuilder ಪ್ಯಾಟರ್ನ್**
   - `.set_start_executor()` ಬಳಸಿ ಪ್ರವೇಶ ಬಿಂದುವನ್ನು ವ್ಯಾಖ್ಯಾನಿಸಿ
   - `.add_edge(from, to, condition=...)` ಬಳಸಿ ಷರತ್ತು ಆಧಾರಿತ ಮಾರ್ಗವನ್ನು ಹೊಂದಿಸಿ
   - `.build()` ಅನ್ನು ಕರೆದು ವರ್ಕ್‌ಫ್ಲೋವನ್ನು ಅಂತಿಮಗೊಳಿಸಿ

2. **ಷರತ್ತು ಆಧಾರಿತ ಮಾರ್ಗ ನಿರ್ಧಾರ**
   - ಷರತ್ತು ಕಾರ್ಯಗಳು `AgentExecutorResponse` ಅನ್ನು ಪರಿಶೀಲಿಸುತ್ತವೆ
   - ರಚನಾತ್ಮಕ ಔಟ್‌ಪುಟ್‌ಗಳನ್ನು ವಿಶ್ಲೇಷಿಸಿ ಮಾರ್ಗ ನಿರ್ಧಾರಗಳನ್ನು ತೆಗೆದುಕೊಳ್ಳಿ
   - ಎಡ್ಜ್ ಸಕ್ರಿಯಗೊಳಿಸಲು `True` ಅನ್ನು ಹಿಂತಿರುಗಿಸಿ, ಅದನ್ನು ಬಿಟ್ಟುಹೋಗಲು `False` ಅನ್ನು ಹಿಂತಿರುಗಿಸಿ

3. **ಟೂಲ್ ಇಂಟಿಗ್ರೇಶನ್**
   - Python ಕಾರ್ಯಗಳನ್ನು AI ಟೂಲ್‌ಗಳಾಗಿ ಪರಿವರ್ತಿಸಲು `@ai_function` ಬಳಸಿ
   - ಏಜೆಂಟ್‌ಗಳು ಅಗತ್ಯವಿದ್ದಾಗ ಟೂಲ್‌ಗಳನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಕರೆಸುತ್ತವೆ
   - ಟೂಲ್‌ಗಳು ಏಜೆಂಟ್‌ಗಳು ವಿಶ್ಲೇಷಿಸಬಹುದಾದ JSON ಅನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತವೆ

4. **ರಚನಾತ್ಮಕ ಔಟ್‌ಪುಟ್‌ಗಳು**
   - ಪ್ರಕಾರ-ಸುರಕ್ಷಿತ ಡೇಟಾ ಹೊರತೆಗೆಯಲು Pydantic ಮಾದರಿಗಳನ್ನು ಬಳಸಿ
   - ಏಜೆಂಟ್‌ಗಳನ್ನು ರಚಿಸುವಾಗ `response_format=MyModel` ಅನ್ನು ಹೊಂದಿಸಿ
   - `Model.model_validate_json()` ಬಳಸಿ ಪ್ರತಿಕ್ರಿಯೆಗಳನ್ನು ವಿಶ್ಲೇಷಿಸಿ

5. **ಕಸ್ಟಮ್ ಎಕ್ಸಿಕ್ಯೂಟರ್‌ಗಳು**
   - `@executor(id="...")` ಬಳಸಿ ವರ್ಕ್‌ಫ್ಲೋ ಘಟಕಗಳನ್ನು ರಚಿಸಿ
   - ಎಕ್ಸಿಕ್ಯೂಟರ್‌ಗಳು ಡೇಟಾವನ್ನು ಪರಿವರ್ತಿಸಬಹುದು ಅಥವಾ ಪಾರ್ಶ್ವ ಪರಿಣಾಮಗಳನ್ನು ನಿರ್ವಹಿಸಬಹುದು
   - `ctx.yield_output()` ಬಳಸಿ ವರ್ಕ್‌ಫ್ಲೋ ಫಲಿತಾಂಶಗಳನ್ನು ಉತ್ಪಾದಿಸಿ

### 🚀 ನೈಜ ಜಗತ್ತಿನ ಅನ್ವಯಿಕೆಗಳು:

- **ಪ್ರಯಾಣ ಬುಕ್ಕಿಂಗ್**: ಲಭ್ಯತೆ ಪರಿಶೀಲಿಸಿ, ಪರ್ಯಾಯಗಳನ್ನು ಸೂಚಿಸಿ, ಆಯ್ಕೆಗಳನ್ನು ಹೋಲಿಸಿ
- **ಗ್ರಾಹಕ ಸೇವೆ**: ಸಮಸ್ಯೆಯ ಪ್ರಕಾರ, ಭಾವನೆ, ಆದ್ಯತೆಯ ಆಧಾರದ ಮೇಲೆ ಮಾರ್ಗ ನಿರ್ಧಾರ
- **ಇ-ಕಾಮರ್ಸ್**: ಇನ್‌ವೆಂಟರಿ ಪರಿಶೀಲಿಸಿ, ಪರ್ಯಾಯಗಳನ್ನು ಸೂಚಿಸಿ, ಆರ್ಡರ್‌ಗಳನ್ನು ಪ್ರಕ್ರಿಯೆಗೊಳಿಸಿ
- **ಕಂಟೆಂಟ್ ಮೋಡರೇಶನ್**: ವಿಷಕಾರಿ ಅಂಕೆ, ಬಳಕೆದಾರರ ಫ್ಲ್ಯಾಗ್‌ಗಳ ಆಧಾರದ ಮೇಲೆ ಮಾರ್ಗ ನಿರ್ಧಾರ
- **ಅನುಮೋದನೆ ವರ್ಕ್‌ಫ್ಲೋಗಳು**: ಮೊತ್ತ, ಬಳಕೆದಾರರ ಪಾತ್ರ, ಅಪಾಯ ಮಟ್ಟದ ಆಧಾರದ ಮೇಲೆ ಮಾರ್ಗ ನಿರ್ಧಾರ
- **ಬಹು ಹಂತದ ಪ್ರಕ್ರಿಯೆ**: ಡೇಟಾ ಗುಣಮಟ್ಟ, ಸಂಪೂರ್ಣತೆಯ ಆಧಾರದ ಮೇಲೆ ಮಾರ್ಗ ನಿರ್ಧಾರ

### 📚 ಮುಂದಿನ ಹಂತಗಳು:

- ಹೆಚ್ಚು ಸಂಕೀರ್ಣ ಷರತ್ತುಗಳನ್ನು ಸೇರಿಸಿ (ಬಹು ಮಾನದಂಡಗಳು)
- ವರ್ಕ್‌ಫ್ಲೋ ಸ್ಥಿತಿಯ ನಿರ್ವಹಣೆಯೊಂದಿಗೆ ಲೂಪ್‌ಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸಿ
- ಪುನಃ ಬಳಸಬಹುದಾದ ಘಟಕಗಳಿಗಾಗಿ ಉಪ-ವರ್ಕ್‌ಫ್ಲೋಗಳನ್ನು ಸೇರಿಸಿ
- ನೈಜ API‌ಗಳೊಂದಿಗೆ (ಹೋಟೆಲ್ ಬುಕ್ಕಿಂಗ್, ಇನ್‌ವೆಂಟರಿ ಸಿಸ್ಟಮ್‌ಗಳು) ಏಕೀಕರಣಗೊಳಿಸಿ
- ದೋಷ ನಿರ್ವಹಣೆ ಮತ್ತು ಬ್ಯಾಕಪ್ ಮಾರ್ಗಗಳನ್ನು ಸೇರಿಸಿ
- ನಿರ್ಮಿತ ದೃಶ್ಯೀಕರಣ ಸಾಧನಗಳೊಂದಿಗೆ ವರ್ಕ್‌ಫ್ಲೋಗಳನ್ನು ದೃಶ್ಯೀಕರಿಸಿ


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸಮೀಕ್ಷೆ**:  
ಈ ದಾಖಲೆ AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಖಚಿತತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಡ್ಡೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅರ್ಥೈಸುವಿಕೆಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
